# Referance
1- https://chatgpt.com/share/67b90811-cfa0-8008-8b81-de02aac5e707

# pip install

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
import torch
import numpy as np
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments




In [3]:

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
distilbert_param = model.num_parameters()/1000000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_param)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'


In [5]:
test =  "This is a great [MASK]."

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
model.config

DistilBertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.48.3",
  "vocab_size": 30522
}

In [8]:
inputs = tokenizer(test, return_tensors="pt")
result = model(**inputs)
logits = result.logits
masked_token_index = torch.where(inputs.input_ids[0] == tokenizer.mask_token_id)
masked_token = logits[0,masked_token_index,:]
top_k_token = torch.topk(masked_token,k=5,dim=1).indices[0].tolist()

In [9]:
inputs

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 2307,  103, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
tokenizer.is_fast

True

In [11]:
inputs.word_ids(batch_index=0)

[None, 0, 1, 2, 3, 4, 5, None]

In [12]:
for token in top_k_token:
    print(f"'>>> {test.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


## the model predction based on English Wikipedia, which is trained on it

## Let’s see how we can change this domain to something a bit more niche — highly polarized movie reviews!

# dataset

In [13]:
dataset = load_dataset("imdb")


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [15]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

In [16]:
for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")


'>>> Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'
'>>> Label: 1'

'>>> Review: This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stu

### Although we won’t need the labels for language modeling, we can already see that a 0 denotes a negative review, while a 1 corresponds to a positive one.

In [17]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [
            result.word_ids(i) for i in range(len(result["input_ids"]))
        ]
    return result

In [18]:
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [19]:
#show tokenizer max_legnth
tokenizer.model_max_length

512

# Preprocessing

#### a common preprocessing step is to concatenate all the examples and then split the whole corpus into chunks of equal size. This is quite different from our usual approach, where we simply tokenize individual examples. Why concatenate everything together? The reason is that individual examples might get truncated if they’re too long, and that would result in losing information that might be useful for the language modeling task!

In [20]:
tokenized_sample = tokenized_datasets["train"][:3]
for key, token in enumerate(tokenized_sample["input_ids"]):
    print(f"Review {key} length:  {len(token)}")

Review 0 length:  363
Review 1 length:  304
Review 2 length:  133


In [21]:
context_length = 128

In [22]:
def tokenize_and_chunk(examples):
    all_input_ids = []

    # Iterate through each 'text' in the batch
    for text in examples["text"]:
        data_tokenized = tokenizer(text)
        input_ids = data_tokenized["input_ids"]
        all_input_ids.extend(input_ids)
        all_input_ids.append(tokenizer.eos_token_id)

    chunks = []
    for idx in range(0, len(all_input_ids), context_length):
        chunk = all_input_ids[idx : idx + context_length]
        chunks.append(chunk)

    # Convert chunks into tensors with padding
    model_inputs = tokenizer.pad(
        {"input_ids": chunks},
        padding="max_length",
        max_length=context_length,
        return_attention_mask=True
    )

    # Labels are the same as input_ids for causal language modeling
    model_inputs["labels"] = model_inputs["input_ids"].copy()

    return model_inputs  # This returns a dictionary with 'input_ids', 'attention_mask', and 'labels'


In [23]:

# Remove unnecessary columns and keep only 'input_ids' before chunking
tokenized_datasetss =dataset.map(tokenize_and_chunk, batched=True, remove_columns=["text", "label"])
tokenized_datasetss

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 61509
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 60125
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 123401
    })
})

In [24]:
tokenized_datasetss

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 61509
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 60125
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 123401
    })
})

# or use anthore function

In [25]:
def group_texts(examples):
  concatenated_examples = {k: sum(examples[k],[])for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  total_length = (total_length//context_length)*context_length
  result = {
      k: [t[i : i + context_length] for i in range (0,total_length,context_length)]
      for k , t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result

In [26]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [27]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it ' s not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

In [28]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it ' s not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

### We’re missing a key step: inserting [MASK] tokens at random positions in the inputs! Let’s see how we can do this on the fly during fine-tuning using a special data collator.

# Fine-tuning DistilBERT with the Trainer API

In [29]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm_probability=0.15)

In [30]:
samples = [lm_datasets["train"][i] for i in range(3)]
for sample in samples:
    _ = sample.pop("word_ids")

for item in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(item)}'")


'>>> [CLS] i rented i am curious - yellow from my video [MASK] because of [MASK] [MASK] controversy that surrounded it when it was first [MASK] in 1967. i also [MASK] that at [MASK] it was [MASK] by u [MASK] s. customs if it [MASK] tried [MASK] enter [MASK] country, therefore being a [MASK] of films considered " controversial explore i really [MASK] [MASK] see this for myself. < [MASK] / > < br / > the plot hammering centered around a young swedish drama student named lena who wants to learn everything she can about life. in particular she wants to focus her [MASK]s to making some sort of documentary on what the average swede thought about certain political issues such'

'>>> as the vietnam war [MASK] race issues in宮 united states. in between asking politicians and [MASK] denizens of stockholm [MASK] their opinions [MASK] politics, she has sex with her drama teacher [MASK] classmates [MASK] and married battling. < br / > < br / > what kills me about i am curious - [MASK] is that 40 ye

### so we’ll first downsample the size of the training set to a few thousand examples. Don’t worry, we’ll still get a pretty decent language model! A quick way to downsample a dataset in 🤗 Datasets is via the Dataset.train_test_split() function

In [31]:
train_size = 10_000
test_size = int(0.1 * train_size)

down_sampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)


In [32]:
down_sampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [33]:
from huggingface_hub import notebook_login

notebook_login()

# specify the arguments for the Trainer

In [37]:
batch_size = 64
#  Show the training loss with every epoch
logging_steps = len(down_sampled_dataset["train"]) // batch_size
model_name= model_checkpoint.split('/')[-1]

In [38]:
training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = down_sampled_dataset["train"],
    eval_dataset = down_sampled_dataset["test"],
    data_collator = data_collator,

)

# to compute metrics

In [40]:
import math
import numpy as np


In [41]:
eval_result = trainer.evaluate()
print (f"Perplexity: {math.exp(eval_result['eval_loss']):.2f}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: a48104046 (a48104046-dfas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Perplexity: 21.94


# after Training

In [42]:
trainer.train()

Step,Training Loss,Validation Loss,Model Preparation Time
156,2.683800,2.509699,0.001800
312,2.587800,2.449211,0.001800
468,2.528800,2.448768,0.001800


TrainOutput(global_step=471, training_loss=2.5997680554724045, metrics={'train_runtime': 159.389, 'train_samples_per_second': 188.219, 'train_steps_per_second': 2.955, 'total_flos': 994208670720000.0, 'train_loss': 2.5997680554724045, 'epoch': 3.0})

In [43]:
eval_result = trainer.evaluate()
print (f"Perplexity: {math.exp(eval_result['eval_loss']):.2f}")

Perplexity: 12.05


In [44]:
trainer.push_to_hub()

events.out.tfevents.1740179526.552c268c34b8.654.1:   0%|          | 0.00/425 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ah7med/distilbert-base-uncased-finetuned-imdb/commit/6291d6266f654d7e6ea430a261bebecdd6512d06', commit_message='End of training', commit_description='', oid='6291d6266f654d7e6ea430a261bebecdd6512d06', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ah7med/distilbert-base-uncased-finetuned-imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='Ah7med/distilbert-base-uncased-finetuned-imdb'), pr_revision=None, pr_num=None)

## Fine-tuning DistilBERT with 🤗 Accelerate using Pytorch

#### we saw that DataCollatorForLanguageModeling also applies random masking with each evaluation, so we’ll see some fluctuations in our perplexity scores with each training run. One way to eliminate this source of randomness is to apply the masking once on the whole test set, and then use the default data collator in 🤗 Transformers to collect the batches during evaluation. To see how this works, let’s implement a simple function that applies the masking on a batch, similar to our first encounter with DataCollatorForLanguageModeling:

In [46]:
def insert_random_mask(batch):
  features = [dict(zip(batch,t)) for t in zip(*batch.values())]
  masked_inputss = data_collator(features)
  # Create a new "masked" column for each key in the masked_input dictionary
  return {"masked_" + k: v.numpy() for k, v in masked_inputss.items()}

In [49]:
down_sampled_dataset = down_sampled_dataset.remove_columns(["word_ids"])

In [50]:
down_sampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [51]:
eval_dataset = down_sampled_dataset["test"].map(insert_random_mask, batched=True, remove_columns=list(down_sampled_dataset["test"].features))

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [52]:
eval_dataset

Dataset({
    features: ['masked_input_ids', 'masked_attention_mask', 'masked_labels'],
    num_rows: 1000
})

In [56]:
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

# Data_Loader

In [71]:
from torch.utils.data import DataLoader
from transformers import default_data_collator
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler

In [58]:
batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataset = DataLoader(
    eval_dataset,
    batch_size=batch_size,
    collate_fn=default_data_collator,
)

In [59]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [61]:
optimizer = AdamW(model.parameters(), lr=5e-5)

# Accelerator object

In [68]:
accelerator = Accelerator()

In [69]:

model,optimizer,train_dataloader,eval_dataset = accelerator.prepare(
    model,optimizer,train_dataloader,eval_dataset)

## Now that our model, optimizer, and dataloaders are configured,

# learning rate scheduler

In [72]:
num_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_update_steps_per_epoch * num_epochs

In [73]:
lr_scheduler = get_scheduler(
    name= "linear",
    optimizer = optimizer,
    num_warmup_steps= 0,
    num_training_steps = num_training_steps
)

In [74]:
from huggingface_hub import get_full_repo_name

model_name = "distilbert-base-uncased-finetuned-accelerate_Movie_Witcher"
repo_name = get_full_repo_name(model_name)
repo_name

'Ah7med/distilbert-base-uncased-finetuned-accelerate_Movie_Witcher'

In [75]:
from huggingface_hub import Repository

output_dir = model_name
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Ah7med/distilbert-base-uncased-finetuned-accelerate_Movie_Witcher into local empty directory.


# full training and evaluation loop

In [77]:
from tqdm.auto import tqdm
import torch
import math

In [84]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
    outputs= model(**batch)
    loss = outputs.loss
    accelerator.backward(loss)
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  model.eval()
  losses = []
  for step, batch  in enumerate(eval_dataset):
    with torch.no_grad():
      outputs= model(**batch)
    loss = outputs.loss
    losses.extend(accelerator.gather(loss.repeat(batch_size)).tolist())
  losses = torch.tensor(losses)
  losses = losses[:len(eval_dataset)]
  try :
    perplexity = math.exp(torch.mean(losses))
  except OverflowError:
    perplexity = float("inf")
  print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

  accelerator.wait_for_everyone()
  unwaraped_model = accelerator.unwrap_model(model)
  unwaraped_model.save_pretrained(output_dir,save_function=accelerator.save)
  if accelerator.is_main_process:
    tokenizer.save_pretrained(output_dir)
    repo.push_to_hub(
        commit_message=f"Training in progress epoch {epoch}", blocking=False
    )

  0%|          | 0/471 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 10.051303252191433
>>> Epoch 1: Perplexity: 10.051303252191433
>>> Epoch 2: Perplexity: 10.051303252191433


In [85]:
 repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

# Using our fine-tuned model

In [86]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="Ah7med/distilbert-base-uncased-finetuned-accelerate_Movie_Witcher"
)

config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [88]:
preds = mask_filler(test)

In [94]:
for sample in preds:
  print(f">>> {sample['sequence']}")

>>> this is a great film.
>>> this is a great movie.
>>> this is a great idea.
>>> this is a great one.
>>> this is a great job.


# Neat — our model has clearly adapted its weights to predict words that are more strongly associated with movies

In [ ]:
# https://colab.research.google.com/drive/1vTMhER5NpiqxCX8lth6wm50coN_CmDWx#scrollTo=YeRK6XRiC9Vv